# Esercizio 3 - Hanks

- Il verbo scelto è **Play**
- Si estrae dal corpus **Brown** le istanze che contengono il verbo **Play**
- Si effettua parsing e disambiguazione delle frasi estratte
- Si usano i super sensi di WordNet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Si aggregano i risultati, si calcolano le frequenze, e si stampano i cluster semantici ottenuti

In [5]:
import nltk
from nltk.corpus import brown
from nltk.wsd import lesk
from nltk.corpus import wordnet as wn
import spacy

### Estrai le frasi che contengono il verbo Play dal corpus Brown

In [37]:
nlp = spacy.load('en_core_web_sm')
# verb = ['study', 'studies', 'studied']
verb = ['play', 'plays', 'played']
sentences = [sent for sent in brown.sents() if verb[0] in sent or verb[1] in sent or verb[2] in sent]
print(len(sentences))
for i in range(10):
    print(' '.join(sentences[i]))

357
The effectiveness of the governor in clearing up some of the inconsistencies revolving about the sales tax bill may play a part in determining whether it can muster the required two-thirds vote .
The Philadelphia Transportation Co. is investigating the part its organization played in reviewing the project .
Bill Kunkel , Bob Hartman and Ed Keegan did the mound chores for the club down from West Palm Beach to play the game before 767 paying customers in Miami Stadium .
When Robinson tried to stretch his blow into a triple , he was cut down in a close play at third , Tuttle to Andy Carey .
Then Throneberry rapped into a fast double play .
-- A Texas halfback who doesn't even know the team's plays , Eldon Moritz , ranks fourth in Southwest Conference scoring after three games .
Although he never gets to play while the clock is running , he gets a big kick -- several every Saturday , in fact -- out of football .
But ask coach Darrell Royal what position he plays and you'll get the quic

### Parsing delle frasi estratte per trovare il soggetto e l'oggetto del verbo Play

Per ogni frase estratta dal corpus cerco il soggetto e l'oggetto del verbo **Play** e li salvo in una lista di tuple.  
Per farlo uso la libreria **spaCy** che mi permette di cercare il soggetto e l'oggetto di un verbo in una frase tramite `token.dep_ == 'nsubj'` e `token.dep_ == 'dobj'`.  
Tramite `token.head.text in verb` posso inoltre cercare se il soggetto e l'oggetto trovati sono effettivamente legati al verbo **Play**.  
Infine con `token.dep_ == 'ROOT'` posso sapere se il verbo **Play** è la radice della frase, senza questo controllo la ricerca del soggetto e dell'oggetto potrebbe portare a risultati non corretti, tipo pronomi o quant'altro.  
Mi sono accorto di ciò interpretando i risultati ottenuti.

In [46]:
# Parse the sentences using Spacy to extract the subject and object of the verb love
def get_subject_object(sentence):
    # len = 0
    triples = []
    for sent in sentences:
        sub = ''
        obj = ''
        doc = nlp(' '.join(sent))
        isRoot = False
        for token in doc:
            if token.dep_ == 'nsubj' and token.head.text in verb:
                sub = token.text
            if token.dep_ == 'dobj' and token.head.text in verb:
                obj = token.text
            # if token.dep_ == 'ROOT' and token.text in verb:
            #     isRoot = True
        if sub != '' and obj != '':
            # len += 1
            triples.append((sub, obj, sent))
    return triples

print(get_subject_object(sentences)[:3])

[('effectiveness', 'part', ['The', 'effectiveness', 'of', 'the', 'governor', 'in', 'clearing', 'up', 'some', 'of', 'the', 'inconsistencies', 'revolving', 'about', 'the', 'sales', 'tax', 'bill', 'may', 'play', 'a', 'part', 'in', 'determining', 'whether', 'it', 'can', 'muster', 'the', 'required', 'two-thirds', 'vote', '.']), ('he', 'position', ['But', 'ask', 'coach', 'Darrell', 'Royal', 'what', 'position', 'he', 'plays', 'and', "you'll", 'get', 'the', 'quick', 'response', ',', '``', 'place-kicker', "''", '.']), ('Nelson', 'football', ['``', 'Nelson', 'played', 'magnificent', 'football', "''", ',', 'Meek', 'praised', '.'])]


### Disambiguazione delle coppie soggetto-oggetto

Per ogni coppia soggetto-oggetto estratta dal corpus, cerco il super-senso di WordNet che meglio descrive il soggetto e l'oggetto.

In [50]:
# Get the super senses of the subject and object
def get_super_sense(triples):
    super_senses = []
    for triple in triples:
        sub = triple[0]
        obj = triple[1]
        sent = triple[2]
        sub_sense = lesk(sent, sub)
        obj_sense = lesk(sent, obj)
        super_senses.append((sub_sense.lexname() if sub_sense else '', obj_sense.lexname() if obj_sense else ''))
    return super_senses

print(get_super_sense(get_subject_object(sentences)))

[('noun.attribute', 'noun.act'), ('noun.substance', 'noun.state'), ('noun.act', 'noun.artifact'), ('', 'noun.animal'), ('noun.group', 'noun.act'), ('', 'noun.act'), ('noun.substance', 'noun.act'), ('noun.group', 'noun.act'), ('', 'noun.cognition'), ('noun.substance', 'verb.contact'), ('', 'adv.all'), ('noun.artifact', 'noun.cognition'), ('noun.substance', 'noun.act'), ('noun.group', 'verb.creation'), ('', 'noun.object'), ('noun.cognition', 'noun.artifact'), ('noun.substance', 'verb.contact'), ('noun.person', 'noun.communication'), ('noun.group', 'noun.act'), ('noun.location', ''), ('verb.contact', 'noun.communication'), ('noun.substance', 'noun.communication'), ('noun.group', 'noun.act'), ('noun.substance', 'noun.person'), ('', 'noun.act'), ('noun.group', 'noun.process'), ('', ''), ('', ''), ('noun.substance', 'noun.person'), ('noun.group', 'noun.animal'), ('', ''), ('', ''), ('noun.group', 'noun.person'), ('noun.group', 'noun.communication'), ('', ''), ('noun.person', 'noun.act'), ('n